''' This program runs a dynamic topic model

#Date: Mar 4, 2019

#Author: Carly Knight

#https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/dtm_example.ipynb
#https://stackoverflow.com/questions/50413059/dynamic-topic-modeling-with-gensim-which-code'''


## set up

In [1]:
import logging
import os
from gensim import corpora, utils
from gensim.models.wrappers.dtmmodel import DtmModel
import numpy as np
import pandas as pd
from gensim.corpora import Dictionary
from collections import defaultdict



### dtm

In [2]:
# you can also copy the path down directly. Change this variable to your DTM executable before running.
dtm_path = "/usr/local/bin/dtm-darwin64"

# Open corpus

In [3]:
file_loc = "/Users/carlyknight/Documents/Data/Annual Report/report_paragraphs/future_texts/"

#limit to over 1930
metadata = pd.read_csv(file_loc + "metadata_futureperfect.csv") 
metadata = metadata[(metadata['Year'] >= 1930)]
metadata['text_filename'] = metadata['Filename'].str.replace(".xml", ".txt")

#pick relevant files
textfiles = [i for i in metadata['text_filename'].values.tolist()]

In [4]:
#open files
for index, row in metadata.iterrows():
    txtfile= row['Filename'].replace(".xml", ".txt")
    #read in text file and turn into new variable
    with open(file_loc + txtfile, 'r') as f:
        metadata.loc[index,'text']= f.read()
        f.close()

In [5]:
#metadata.shape (36479,17)

In [6]:
#remove puncutation
#metadata['text'] = metadata['text'].str.replace(r'[^\w\s]+', '')

#remove numbers
#metadata['text'] = metadata['text'].str.replace(r'[\d]+', '')

#remove extraspaces
#metadata['text'] = metadata['text'].str.replace(r'\s+', ' ')

#turn to text
documents = metadata['text'].tolist()

## Preprocess

In [7]:
from gensim.parsing.preprocessing import preprocess_string

#processed corpus: removes punctuation, whitespaces, stopwords, stems, numbes
processed_corpus = [preprocess_string(document) for document in documents] 

#remove low frequency terms (terms with fewer than 20) (brings it down to 35,244 from 953,010 tokens)
frequency = defaultdict(int)

for text in processed_corpus:
    for token in text:
        frequency[token] += 1

processed_corpus = [
    [token for token in text if frequency[token] > 10]
    for text in processed_corpus
]

#dictionary
dictionary = Dictionary(processed_corpus)

#filter extremes: get rid of words that appear in no fewer than 5 documents
#dictionary.filter_extremes(no_below=5)
#len(dictionary) #46,729

#Bag of Words
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

#Time slice
counts = metadata[['Year', "Filename"]].groupby(['Year']).agg(['count'])
timeslice=[i[0] for i in counts.values.tolist()]

# DTM

In [12]:
model = DtmModel(dtm_path, bow_corpus, timeslice, num_topics=60,
                 id2word=dictionary, initialize_lda=True)

In [9]:
topics = model.show_topic(topicid=19, time=10, topn=30)
topics

[(0.013394590199554154, 'railroad'),
 (0.011332864069375165, 'servic'),
 (0.011084563635882283, 'new'),
 (0.00991507591547782, 'car'),
 (0.009347388276176673, 'increas'),
 (0.008847047950897417, 'oper'),
 (0.008818846784891152, 'transport'),
 (0.008429487793592722, 'compani'),
 (0.008364064260096485, 'year'),
 (0.006200515017458958, 'passeng'),
 (0.00605686167668141, 'line'),
 (0.00603141802529653, 'revenu'),
 (0.00576494443124274, 'traffic'),
 (0.005715563490744544, 'aircraft'),
 (0.0056702412424613465, 'pacif'),
 (0.005439196582400276, 'freight'),
 (0.005365936276850821, 'equip'),
 (0.00526788050937191, 'rate'),
 (0.004787691269615274, 'addit'),
 (0.004732856900816652, 'air'),
 (0.004540767684385692, 'mile'),
 (0.004496287937806671, 'tax'),
 (0.004442185291977374, 'airlin'),
 (0.004292565792113813, 'commiss'),
 (0.004214059324835335, 'coal'),
 (0.004084125436405238, 'rail'),
 (0.003762931509631709, 'northern'),
 (0.003754380929269193, 'industri'),
 (0.0035955058373107916, 'effect'),


In [19]:
#https://towardsdatascience.com/topic-modeling-with-gensim-a5609cefccc

# save

In [13]:
import pickle

In [14]:
f = open('/Users/carlyknight/Documents/Data/FuturePerfect/dynamic_topic_models/topic60.pkl', 'wb') 
pickle.dump(model, f , protocol=4)          
f.close()                 